# Import Libraries

In [2]:
# --- Standard ---
import os, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- Torch ---
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F

# --- Vision ---
from torchvision import transforms
from PIL import Image

# --- Grad-CAM ---
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image

# --- Google Drive ---
from google.colab import drive

ModuleNotFoundError: No module named 'pytorch_grad_cam'

# Find Paths

In [ ]:
drive.mount('/content/drive')
SPLITS_DIR = "/content/drive/MyDrive/neuro-imaging/splits"
OUT_DIR    = "/content/drive/MyDrive/neuro-imaging/models"

# Open Class Information

In [ ]:
with open(os.path.join(SPLITS_DIR, "class_to_idx.json"), "r") as f:
    class_to_idx = json.load(f)
idx_to_class = {v:k for k,v in class_to_idx.items()}
num_classes = len(class_to_idx)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Model

In [ ]:
import timm
ckpt = torch.load(os.path.join(OUT_DIR, "best.pt"), map_location=device)
model = timm.create_model("efficientnet_b0", pretrained=False, num_classes=num_classes)
model.load_state_dict(ckpt["model_state_dict"])
model.eval().to(device)

# Transform

In [3]:
IMG_SIZE = 224
val_tfms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

# Picking Target Layer for GradCam

In [ ]:
# EfficientNetB0 -> final conv layer is model.blocks[-1]
target_layers = [model.blocks[-1]]

cam = GradCAM(model=model, target_layers=target_layers, use_cuda=(device.type=="cuda"))

# Sample GradCam on a Few Images

In [8]:
SAMPLES = 5
sample_paths = []
with open(os.path.join(SPLITS_DIR, "val.txt")) as f:
    sample_paths = [ln.strip() for ln in f][:SAMPLES]

for path in sample_paths:
    img = Image.open(path).convert("RGB")
    input_tensor = val_tfms(img).unsqueeze(0).to(device)

    # Run CAM
    grayscale_cam = cam(input_tensor=input_tensor)[0, :]
    cam_image = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

    # Plot
    plt.figure(figsize=(6,3))
    plt.subplot(1,2,1)
    plt.imshow(rgb_img)
    plt.title("Original")
    plt.axis("off")

    plt.subplot(1,2,2)
    plt.imshow(cam_image)
    plt.title("Grad-CAM")
    plt.axis("off")

    plt.suptitle(os.path.basename(path))
    plt.show()

NameError: name 'SPLITS_DIR' is not defined